# Loading and Exploring the SpaceX Dataset using sqlite3

In [58]:
#!pip install ipython-sql
#!pip install ipython-sql prettytable

In [2]:
!pip install sqlalchemy

## Connect to the database

In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect('database/my_data1.db')
cur = con.cursor()

In [5]:
import pandas as pd

In [7]:
# Instantiate the database
%sql sqlite:///database/my_data1.db

In [8]:
# Read dataFrame
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Connect data to the Database
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False, method="multi")

101

In [9]:
# Remove blank rows from the table 
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///database/my_data1.db
Done.


[]

In [11]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null;

 * sqlite:///database/my_data1.db
Done.


[]

In [12]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///database/my_data1.db
Done.


name
SPACEXTBL
SPACEXTABLE


In [13]:
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('SPACEXTABLE');

 * sqlite:///database/my_data1.db
Done.


name,type,length(type)
Date,TEXT,4
Time (UTC),TEXT,4
Booster_Version,TEXT,4
Launch_Site,TEXT,4
Payload,TEXT,4
PAYLOAD_MASS__KG_,INT,3
Orbit,TEXT,4
Customer,TEXT,4
Mission_Outcome,TEXT,4
Landing_Outcome,TEXT,4


In [16]:
%sql select distinct(Launch_Site) from SPACEXTABLE;

 * sqlite:///database/my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [19]:
%sql select * from SPACEXTABLE where Launch_Site like 'CCA%' limit 5;

 * sqlite:///database/my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [28]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTABLE where Customer='NASA (CRS)';

 * sqlite:///database/my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


In [30]:
%sql select round(avg(PAYLOAD_MASS__KG_), 2) from SPACEXTABLE where Booster_Version LIKE 'F9 v1.1%';

 * sqlite:///database/my_data1.db
Done.


"round(avg(PAYLOAD_MASS__KG_), 2)"
2534.67


In [31]:
%sql select min(Date) from SPACEXTABLE;

 * sqlite:///database/my_data1.db
Done.


min(Date)
2010-06-04


In [35]:
%sql select Booster_Version	from SPACEXTABLE where Landing_Outcome = 'Success' and PAYLOAD_MASS__KG_ between 4000 AND 6000;

 * sqlite:///database/my_data1.db
Done.


Booster_Version
F9 B5 B1046.2
F9 B5 B1047.2
F9 B5 B1046.3
F9 B5 B1048.3
F9 B5 B1051.2
F9 B5B1060.1
F9 B5 B1058.2
F9 B5B1062.1


In [38]:
%sql select count(Mission_Outcome) from SPACEXTABLE;

 * sqlite:///database/my_data1.db
Done.


count(Mission_Outcome)
101


In [42]:
%sql select * from SPACEXTABLE where PAYLOAD_MASS__KG_ IN (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE); 

 * sqlite:///database/my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2019-11-11,14:56:00,F9 B5 B1048.4,CCAFS SLC-40,"Starlink 1 v1.0, SpaceX CRS-19",15600,LEO,SpaceX,Success,Success
2020-01-07,2:33:00,F9 B5 B1049.4,CCAFS SLC-40,"Starlink 2 v1.0, Crew Dragon in-flight abort test",15600,LEO,SpaceX,Success,Success
2020-01-29,14:07:00,F9 B5 B1051.3,CCAFS SLC-40,"Starlink 3 v1.0, Starlink 4 v1.0",15600,LEO,SpaceX,Success,Success
2020-02-17,15:05:00,F9 B5 B1056.4,CCAFS SLC-40,"Starlink 4 v1.0, SpaceX CRS-20",15600,LEO,SpaceX,Success,Failure
2020-03-18,12:16:00,F9 B5 B1048.5,KSC LC-39A,"Starlink 5 v1.0, Starlink 6 v1.0",15600,LEO,SpaceX,Success,Failure
2020-04-22,19:30:00,F9 B5 B1051.4,KSC LC-39A,"Starlink 6 v1.0, Crew Dragon Demo-2",15600,LEO,SpaceX,Success,Success
2020-06-04,1:25:00,F9 B5 B1049.5,CCAFS SLC-40,"Starlink 7 v1.0, Starlink 8 v1.0",15600,LEO,"SpaceX, Planet Labs",Success,Success
2020-09-03,12:46:14,F9 B5 B1060.2,KSC LC-39A,"Starlink 11 v1.0, Starlink 12 v1.0",15600,LEO,SpaceX,Success,Success
2020-10-06,11:29:34,F9 B5 B1058.3,KSC LC-39A,"Starlink 12 v1.0, Starlink 13 v1.0",15600,LEO,SpaceX,Success,Success
2020-10-18,12:25:57,F9 B5 B1051.6,KSC LC-39A,"Starlink 13 v1.0, Starlink 14 v1.0",15600,LEO,SpaceX,Success,Success


In [57]:
%sql select substr(Date, 6, 2) as month_2015, Landing_Outcome, Booster_Version, Launch_site from SPACEXTABLE where Landing_Outcome LIKE 'Failure%' AND substr(Date,1,4)='2015';

 * sqlite:///database/my_data1.db
Done.


month_2015,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [54]:
%sql select Landing_Outcome, count(Landing_Outcome) as Number_of_Outcomes from SPACEXTABLE where Date BETWEEN '2010-06-04' AND '2017-03-20' group by Landing_Outcome order by count(Landing_Outcome) desc;

 * sqlite:///database/my_data1.db
Done.


Landing_Outcome,Number_of_Outcomes
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
